# Getting Access Token

In [1]:
import requests
import base64
from requests import post
from requests import get
import json
import pprint 
import pandas as pd
import spotify_api_data_retrieval as api

In [2]:
client_id = "3f610174211649e6930e17f17645f3dd"
client_secret = "dfb8a88954ed405ca67a62d0837fd82f"

token = api.get_token(client_id, client_secret)

{"access_token": "BQDKNPuJ8xWp_yaAg1ccDeaoorz9obsy0ZZ0cJz0DE-T5hAb_642xgiTn8tI6E7LkhgvJ6lZ223EJ6P5ocHC152wgfiH1tnptIYKGuCtQtlUv6u6lLA", "token_type": "Bearer", "expires_in": 3600}


This function only seems to work within this jupyer notebook. An identical function is in the [spotify_api_data_retrieval.py](spotify_api_data_retrieval.py) script but keeps giving an error when I try to use it.

In [5]:
def get_genres_for_playlist(token, playlist_id):
    '''
    This function gets the artist name, ID, and genre information for all the tracks in a playlist returned as a pandas dataframe.
    
    token: bearer token ID
    playlist_id: playlist ID formatted as a string
    '''
    # get track metadata from playlists
    playlist = api.get_playlist(token, playlist_id)
    tracks = playlist["tracks"]["items"]

    # extract track IDs from dictionary and store in list
    tracks_ids = []

    for song in tracks:
        tracks_ids.append(song["track"]["id"])
        
    # get artist IDs for each track in playlist
    artists_ids = []
    
    for track_id in tracks_ids:
        artists_ids.append(api.get_artist_details_from_track(token, track_id)['artist_id'])
        
    # get genres for each artist
    tracks_genre = []
    
    for artist_id in artists_ids:
        tracks_genre.append(api.get_artist_genre(token, artist_id))

    df = pd.DataFrame(tracks_genre)
    
    return(df)

In [4]:
def holy_trinity(token, playlist_id, playlist_name):
    '''
    This function returns three separate .csv files each containing the following information for all the tracks in 
    a given playlist:
    
    1. Audio features
    2. Genres
    3. Artist and track information
    
    token: bearer token ID
    playlist_id: playlist ID formatted as a string
    playlist_name: name of the playlist formatted as a string
    '''
    
    # Audio Features .csv
    audio_features = api.get_audio_features_for_playlist(token, playlist_id)
    api.df_export_csv(audio_features, playlist_name + '_audio_features')
    
    # Genres .csv
    genres = get_genres_for_playlist(token, playlist_id)
    api.df_export_csv(genres, playlist_name + '_genres')
    
    # Track and Artist Info
    track_and_artist_info = api.get_track_and_artist_info_for_playlist(token, playlist_id)
    api.df_export_csv(track_and_artist_info, playlist_name + '_track_and_artist_info')

## Top 50 USA Chart

In [6]:
# get information in .csv files
holy_trinity(token, '37i9dQZEVXbLp5XoPON0wI', 'data/usa/top_songs_usa')

## Top 50 Australia Chart

In [7]:
# get information in .csv files
holy_trinity(token, '37i9dQZEVXbK4fwx2r07XW', 'data/australia/top_songs_australia')

## Top 50 Brazil Chart

In [ ]:
# get information in .csv files
holy_trinity(token, '37i9dQZEVXbKzoK95AbRy9', 'data/brazil/top_songs_brazil')